In [1]:
import matplotlib.pyplot as plt
from sklearn import preprocessing
import random
import numpy as np
import pandas as pd
from sklearn import datasets, svm, cross_validation, tree, preprocessing, metrics
import h5py
import scipy

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_dataset():
    train_dataset = h5py.File('E:/FCI/year 4/second term/NN/labs/neural-network-and-deep-learning-master/lab3/train_catvnoncat.hdf5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('E:/FCI/year 4/second term/NN/labs/neural-network-and-deep-learning-master/lab3/test_catvnoncat.hdf5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes=load_dataset()

In [4]:
#print dataset shapes(dimensions)
print("train set x dim= "+str(train_set_x_orig.shape))
print("train set y dim= "+str(train_set_y_orig.shape))
print("test set x dim= "+str(test_set_x_orig.shape))
print("test set y dim= "+str(test_set_y_orig.shape))

train set x dim= (209, 64, 64, 3)
train set y dim= (209,)
test set x dim= (50, 64, 64, 3)
test set y dim= (50,)


In [5]:
#reshape dataset
#train data
train_set_x = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
train_set_y = train_set_y_orig.reshape(1,train_set_y_orig.shape[0])
    
#test data
test_set_x = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T
test_set_y = test_set_y_orig.reshape(1,test_set_y_orig.shape[0])
    

In [6]:

#print dataset shapes(dimensions)
print("train set x dim= "+str(train_set_x.shape))
print("train set y dim= "+str(train_set_y.shape))
print("test set x dim= "+str(test_set_x.shape))
print("test set y dim= "+str(test_set_y.shape))


train set x dim= (12288, 209)
train set y dim= (1, 209)
test set x dim= (12288, 50)
test set y dim= (1, 50)


In [7]:
#normalize data
train_set_x=train_set_x/255
test_set_x=test_set_x/255

In [8]:
def sigmoid(input):
    
    s = 1/(1 + np.exp(-input))
    
    return s

In [9]:
def layers_size(x,y):
    n_x=x.shape[0]
    n_y=y.shape[0]
    return n_x,n_y

In [21]:
def initialzation(n_x,n_h,n_y):

    w1 = np.random.randn(n_h,n_x)
    b1 = np.zeros((n_h,1))
    w2 = np.random.randn(n_y,n_h)
    b2 = np.zeros((n_y,1))
              
    parameters ={"w1": w1,
                "b1": b1,
                "w2": w2,
                "b2": b2}                    
                         
    return parameters

In [22]:
def forwardPropagation(X, parameters):
   
    
    
    w1 = parameters["w1"]
    w2 = parameters["w2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION 
    z1 = np.dot(w1, X) + b1
    
    A1 = np.tanh(z1) 
    
    z2 = np.dot(w2, A1) + b2
    
    A2 =sigmoid(z2) 
                
                
    cache ={    "z1": z1,
                "A1": A1,
                "z2": z2,
                "A2": A2} 
    return cache 


In [23]:
def compute_cost (A2,X,Y):
    m  =  X.shape[1]
    logs= np.multiply(np.log(A2) ,Y) + np.multiply((1-Y),np.log(1-A2))
    cost= 1/m * np.sum(logs)
    cost=np.squeeze(cost)
    
    return cost

In [24]:
def backwardPropagation(X,Y,cache, parameters):

        
    m  =  X.shape[1]
    w1 = parameters["w1"]
    w2 = parameters["w2"]
    A1 = cache["A1"]
    A2 = cache["A2"]
    z1 = cache["z1"]

    # BACKWARD PROPAGATION (TO FIND GRAD)
    dz2 = A2 - Y
    
    dw2 = 1/m * np.dot(dz2, A1.T)
    
    db2= 1/m * np.sum(dz2 , axis=1 , keepdims=True)
    
    dz1 = np.multiply(np.dot(w2.T,dz2) , (1-np.power(A1,2)))
    
    dw1 = 1/m * np.dot(dz1, X.T)
    
    db1 = 1/m * np.sum(dz1, axis=1 , keepdims=True)
    
    
    grads = {"dw1": dw1,
             "db1": db1,
             "dw2": dw2,
             "db2": db2}
    
    return grads

In [25]:
def update_parameters(parameters,grads,learning_rate=1.2):
    
    w1 = parameters["w1"]
    w2 = parameters["w2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
    dw1 = grads["dw1"]
    db1 = grads["db1"]
    dw2 = grads["dw2"]
    db2 = grads["db2"]

    w1 = w1 - learning_rate * dw1
    b1 = b1 - learning_rate * db1
    w2 = w2 - learning_rate * dw2
    b2 = b2 - learning_rate * db2
     
    parameters ={"w1": w1,
                "b1": b1,
                "w2": w2,
                "b2": b2}                    
                         
    return parameters        

In [29]:
def predict(parameters, X):

    w1 = parameters["w1"]
    w2 = parameters["w2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"] 
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    
    
    ### make prediction
    A1 = np.tanh(np.dot(w1, X) + b1)
    
    A2 = sigmoid(np.dot(w2, A1) + b2)
    ### END CODE HERE ###

    for i in range(A2.shape[1]):        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        if A2[0,i] >= 0.5:
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
    
    
    return Y_prediction

In [32]:
def model(X_train, Y_train, X_test, Y_test,n_h,num_iterations=10000,learning_rate=1.2,print_cost = True):
    
    n_x,n_y = layers_size(X_train,Y_train)

    parameters=initialzation(n_x,n_h,n_y)
    costs=[]
    for i in range(num_iterations):
        
        cache = forwardPropagation(X_train,parameters)
        
        A2 = cache["A2"]
        cost  = compute_cost (A2,X_train,Y_train)
        
        grads = backwardPropagation(X_train,Y_train,cache, parameters)
        
        parameters = update_parameters(parameters,grads,learning_rate)
        
                # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    # Predict test/train set examples 
    Y_prediction_test = predict(parameters, X_test)
    Y_prediction_train = predict(parameters, X_train)

    ### END CODE HERE ###

    # Print train/test accuracy
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "grads" : grads,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [34]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y,4, num_iterations = 2000, learning_rate = 0.5)

Cost after iteration 0: -1.205442
Cost after iteration 100: -0.610390
Cost after iteration 200: -0.595413
Cost after iteration 300: -0.589691
Cost after iteration 400: -0.589327
Cost after iteration 500: -0.580010
Cost after iteration 600: -0.569152
Cost after iteration 700: -0.563038
Cost after iteration 800: -0.561700
Cost after iteration 900: -0.607573
Cost after iteration 1000: -0.543693
Cost after iteration 1100: -0.539725
Cost after iteration 1200: -0.522666
Cost after iteration 1300: -0.598473
Cost after iteration 1400: -0.598350
Cost after iteration 1500: -0.594451
Cost after iteration 1600: -0.589685
Cost after iteration 1700: -0.529758
Cost after iteration 1800: -0.474229
Cost after iteration 1900: -0.493124
train accuracy: 76.07655502392345 %
test accuracy: 54.0 %
